# Где дешевле жить? Предсказание цен в Airbnb - учимся генерировать признаки и интерпретировать результаты модели

In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# Загружаем датасет с помощью Pandas
data = pd.read_csv('AB_NYC_2019.csv')

# # Добавляем в датасет для каждого объекта расстояние до центра Манхеттена
# man_lat = (data[data['neighbourhood_group'] == 'Manhattan']['latitude'].max() - data[data['neighbourhood_group'] == 'Manhattan']['latitude'].min()) / 2
# man_lon = (data[data['neighbourhood_group'] == 'Manhattan']['longitude'].max() - data[data['neighbourhood_group'] == 'Manhattan']['longitude'].min()) / 2
# data['man_dist'] = abs(data['latitude'] - man_lat) + abs(data['longitude'] - man_lon)

# Центр Манхэттена (правильнее взять среднее)
man_center = data[data['neighbourhood_group'] == 'Manhattan'][['latitude', 'longitude']].mean()
man_lat, man_lon = man_center['latitude'], man_center['longitude']
# перевод разницы координат в метры
data['man_dist'] = np.sqrt(
    ((data['latitude'] - man_lat) * 111_000) ** 2 +
    ((data['longitude'] - man_lon) * 85_000) ** 2
).astype(int)

# Кодируем числами категориальные признаки room_type и neighbourhood_group
le = LabelEncoder()
data['room_type'] = le.fit_transform(data['room_type'])
data['neighbourhood'] = le.fit_transform(data['neighbourhood'])
data['neighbourhood_group'] = le.fit_transform(data['neighbourhood_group'])

# Заполняем NoN в reviews_per_month
data['reviews_per_month'] = data['reviews_per_month'].fillna(-1)

# Отрезаем лишние колонки
cols_to_drop = ['id', 'name', 'host_id', 'host_name', 'last_review', 'latitude', 'longitude']
data = data.drop(columns=cols_to_drop)


In [95]:
# Удаляем строки с значением цены = 0
mask = data['price'] == 0
data = data.drop(data[mask].index)

# Убираем строки с отклонениями цены более 2 сигмы (с учётом района, типа и категории жилья)
list_neighbourhood = list(data['neighbourhood'].unique())
list_room_type = list(data['room_type'].unique())

n_sigma = 2

for i in list_neighbourhood:
    for j in list_room_type:
        count = data[(data['neighbourhood'] == i) & (data['room_type'] == j)].shape[0]
        if count != 0:
            mask = (data['neighbourhood'] == i) & (data['room_type'] == j)
            mean = int(data[mask]['price'].mean())

            mask = (data['neighbourhood'] == i) & (data['room_type'] == j)
            std = data[mask]['price'].std()
            std = int(np.nan_to_num(std, nan=0))

            mask = (data['neighbourhood'] == i) & (data['room_type'] == j) & (data['price'] > mean + n_sigma * std)
            data = data.drop(data[mask].index)

            mask = (data['neighbourhood'] == i) & (data['room_type'] == j) & (data['price'] < mean - n_sigma * std)
            data = data.drop(data[mask].index)

In [96]:
# Обучаем регрессор предсказывать цену на датасете
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Свой Scaler который возвращает типы Pandas
class StandardScalerPd(StandardScaler):
    def transform(self, X, *args, **kwargs):
        X_scaled = super().transform(X, *args, **kwargs)
        return pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

    def fit_transform(self, X, *args, **kwargs):
        X_scaled = super().fit_transform(X, *args, **kwargs)
        return pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

#------------------------------------------------------------------------------------------

# Разбиваем датасет на train и test, со стратификацией. Готовим датасеты для Классификатор и для Регрессора
# strat_col = data['neighbourhood'].astype(str) + "_" + data['room_type'].astype(str)

X_train, X_test, y_train, y_test = train_test_split(
    # data.drop(columns=price),
    data.drop('price', axis=1),
    data['price'],
    test_size=0.30,
    random_state=42,
    # stratify=strat_col
)

# Приводим значения признаков к одному масштабу
scaler = StandardScalerPd()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучение из коробки
mod = LGBMRegressor(random_state=42)
# mod = KNeighborsRegressor()
# mod = RandomForestRegressor(random_state=42)
# LGBMRegressor
param_grid = {
    'n_estimators': [100],
    # 'learning_rate': [0.15, 0.2],
    'max_depth': [60],
}
# # KNN
# param_grid = {
#     'n_neighbors': [10, 50, 100],
#     'weights': ['uniform', 'distance'],
# }
# RandomForestRegressor
# param_grid = {
#     'n_estimators': [200],   # число деревьев
#     'max_depth': [20],          # глубина
#     # 'min_samples_split': [2],   # минимальное число объектов для разбиения
#     # 'min_samples_leaf': [1]      # минимальное число объектов в листе
# }
model = GridSearchCV(
    estimator=mod,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=4,         # 4-кратная кросс-валидация
    n_jobs=-1,    # параллельно на всех ядрах
    verbose=2
)

model.fit(X_train, y_train)
y_test_Predictions = model.predict(X_test)

# Посчитаем RMSE
mask1 = y_test < 50
mask2 = (y_test < 150) & (y_test >= 50)
mask3 = (y_test < 600) & (y_test >= 150)
mask4 = y_test >= 600

rmse = np.sqrt(mean_squared_error(y_test, y_test_Predictions))
rmse1 = (np.sqrt(mean_squared_error(y_test[mask1], y_test_Predictions[mask1])) if len(y_test[mask1]) > 0 else 0)
rmse2 = (np.sqrt(mean_squared_error(y_test[mask2], y_test_Predictions[mask2])) if len(y_test[mask2]) > 0 else 0)
rmse3 = (np.sqrt(mean_squared_error(y_test[mask3], y_test_Predictions[mask3])) if len(y_test[mask3]) > 0 else 0)
rmse4 = (np.sqrt(mean_squared_error(y_test[mask4], y_test_Predictions[mask4])) if len(y_test[mask4]) > 0 else 0)

pmin, pmax = y_test.min(), y_test.max()
pmin1, pmax1 = y_test[mask1].min(), y_test[mask1].max()
pmin2, pmax2 = y_test[mask2].min(), y_test[mask2].max()
pmin3, pmax3 = y_test[mask3].min(), y_test[mask3].max()
pmin4, pmax4 = y_test[mask4].min(), y_test[mask4].max()

print('------------------------------------------------------------')
print("Best params:", model.best_params_)
print("Best RMSE:", -model.best_score_)

print('------------------------------------------------------------')
print("RMSE (на рассчитанных значениях Category):", rmse, ', Мин цена: ', pmin, ', Макс цена: ', pmax)
print("RMSE 1 (в ценовом диапазоне до 50 USD):", rmse1, ', Мин цена: ', pmin1, ', Макс цена: ', pmax1)
print("RMSE 2 (в ценовом диапазоне от 50 до 150 USD):", rmse2, ', Мин цена: ', pmin2, ', Макс цена: ', pmax2)
print("RMSE 3 (в ценовом диапазоне от 150 до 600 USD):", rmse3, ', Мин цена: ', pmin3, ', Макс цена: ', pmax3)
print("RMSE 4 (в ценовом диапазоне от 600 USD):", rmse4, ', Мин цена: ', pmin4, ', Макс цена: ', pmax4)


Fitting 4 folds for each of 1 candidates, totalling 4 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1338
[LightGBM] [Info] Number of data points in the train set: 33345, number of used features: 9
[LightGBM] [Info] Start training from score 135.929555
------------------------------------------------------------
Best params: {'max_depth': 60, 'n_estimators': 100}
Best RMSE: 75.94213287802214
------------------------------------------------------------
RMSE (на рассчитанных значениях Category): 70.26437849621675 , Мин цена:  10 , Макс цена:  1200
RMSE 1 (в ценовом диапазоне до 50 USD): 28.18993175207222 , Мин цена:  10 , Макс цена:  49
RMSE 2 (в ценовом диапазоне от 50 до 150 USD): 43.41293973794424 , Мин цена:  50 , Макс цена:  149
RMSE 3 (в ценовом диапазоне от 150 до 600 USD

In [97]:
data.head()

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,man_dist
1,2,127,0,225,1,45,0.38,2,355,1489
2,2,94,1,150,3,0,-1.00,1,365,5615
3,1,41,0,89,1,270,4.64,1,194,8960
4,2,61,0,80,10,9,0.10,1,0,4533
5,2,137,0,200,3,74,0.59,1,129,1930
